In [1]:
import unittest
import torch
import torch.nn as nn

from autoPyTorch.pipeline.base.pipeline import Pipeline
from autoPyTorch.components.networks.image import ConvCusNet
from autoPyTorch.pipeline.nodes.image.network_selector_datasetinfo import NetworkSelectorDatasetInfo
from autoPyTorch.pipeline.nodes.image.create_dataset_info import DataSetInfo

/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
pipeline = Pipeline([
    NetworkSelectorDatasetInfo()
])
dataset_info = DataSetInfo()
selector = pipeline[NetworkSelectorDatasetInfo.get_name()]
selector.add_network("convnet_customize", ConvCusNet)
selector.add_final_activation('none', nn.Sequential())

pipeline_config = pipeline.get_pipeline_config()
pipeline_config["random_seed"] = 42
hyper_config = pipeline.get_hyperparameter_search_space().sample_configuration()

# pipeline.fit_pipeline(hyperparameter_config=hyper_config, pipeline_config=pipeline_config, 
#                         X=torch.rand(1,3,32,32), Y=torch.rand(1, 1))

# Image classification

There are classes for featurized tasks (classification, multi-label classification, regression) and image tasks (classification). You can import them via:

In [12]:
from autoPyTorch import AutoNetImageClassification, AutoNetClassification

In [13]:
# Other imports for later usage
import pandas as pd
import numpy as np
import os as os
import openml
import json

In [25]:
from autoPyTorch.utils.hyperparameter_search_space_update import HyperparameterSearchSpaceUpdates


def get_search_space_updates():
    """
    Search space updates to the task can be added using HyperparameterSearchSpaceUpdates
    Returns:
        HyperparameterSearchSpaceUpdates
    """
    updates = HyperparameterSearchSpaceUpdates()
    updates.append(node_name="NetworkSelector",
                                hyperparameter="convnet_customize:activation",
                                value_range=["relu"])

    updates.append(node_name="NetworkSelector",
                                hyperparameter="convnet_customize:conv_init_filters",
                                value_range=[8, 64],
                                log=True)
    updates.append(node_name="NetworkSelector",
                                hyperparameter="convnet_customize:conv_second_filters",
                                value_range=[8, 64],
                                log=True)

    updates.append(node_name="NetworkSelector",
                                hyperparameter="convnet_customize:conv_third_filters",
                                value_range=[8, 64],
                                log=True)
    return updates

Here are some useful methods provided by the API:

In [26]:
autonet_config = {
    "budget_type" : "epochs",
    "images_shape": [3,32,32],
    }



autonet = AutoNetImageClassification(config_preset="tiny_cs", 
                                     result_logger_dir="logs/",
                                     hyperparameter_search_space_updates=get_search_space_updates(),
                                     **autonet_config
                                    )

In [27]:
# Get the current configuration as dict
current_configuration = autonet.get_current_autonet_config()

# Get the ConfigSpace object with all hyperparameters, conditions, default values and default ranges
hyperparameter_search_space = autonet.get_hyperparameter_search_space()

# Print all possible configuration options 
autonet.print_help()

KeyError: 'NetworkSelectorDatasetInfo:network'

***fit*** is used to search for a good configuration by fitting configurations chosen by the algorithm (by default BOHB). The incumbent configuration is then returned and stored in the class.

We recommend to have a look at the possible configuration options first. Some of the most important options allow you to set the budget type (epochs or time), run id and task id for cluster usage, tensorboard logging, seed and more.

Here we search for a configuration for 300 seconds with 60-100 s time for fitting each individual configuration.
Use the *validation_split* parameter to specify a split size. You can also pass your own validation set
via *X_val* and *Y_val*. Use *log_level="info"* or *log_level="debug"* for more detailed output.

In [8]:
path_to_cifar_csv = os.path.abspath("../../datasets/CIFAR10.csv")

results_fit = autonet.fit(X_train=np.array([path_to_cifar_csv]),
                                 Y_train=np.array([0]),
                                 min_budget=1,
                                 max_budget=2,
                                 max_runtime=1800,
                                 default_dataset_download_dir="./datasets",
                                 images_root_folders=["./datasets"])

Configuration space object:
  Hyperparameters:
    convnet_customize:conv_init_filters, Type: UniformInteger, Range: [8, 64], Default: 36
    convnet_customize:conv_second_filters, Type: UniformInteger, Range: [8, 64], Default: 36
    convnet_customize:conv_third_filters, Type: UniformInteger, Range: [8, 64], Default: 36
    network, Type: Categorical, Choices: {convnet_customize}, Default: convnet_customize
  Conditions:
    convnet_customize:conv_init_filters | network == 'convnet_customize'
    convnet_customize:conv_second_filters | network == 'convnet_customize'
    convnet_customize:conv_third_filters | network == 'convnet_customize'



23:15:30 job (0, 0, 0) failed with exception
Traceback (most recent call last):
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/hpbandster/core/worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker_no_timelimit.py", line 72, in compute
    result, randomstate = self.optimize_pipeline(config, budget, config_id, random.getstate())
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker_no_timelimit.py", line 125, in optimize_pipeline
    raise e
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker_no_timelimit.py", line 119, in optimize_pipeline
    config_id=config_id, working_directory=self.working_directory), random.getstate()
  File "/home/l/liny/an

{'CreateImageDataLoader:batch_size': 91, 'ImageAugmentation:augment': True, 'ImageAugmentation:cutout': True, 'SimpleLearningrateSchedulerSelector:lr_scheduler': 'step', 'SimpleTrainNode:batch_loss_computation_technique': 'mixup', 'ImageAugmentation:autoaugment': False, 'ImageAugmentation:cutout_holes': 3, 'ImageAugmentation:fastautoaugment': True, 'ImageAugmentation:length': 3, 'SimpleLearningrateSchedulerSelector:step:gamma': 0.18228937531801634, 'SimpleLearningrateSchedulerSelector:step:step_size': 5, 'SimpleTrainNode:mixup:alpha': 0.780111187598316, 'LossModuleSelectorIndices:loss_module': 'cross_entropy', 'NetworkSelectorDatasetInfo:network': 'convnet_customize', 'OptimizerSelector:optimizer': 'sgd', 'SimpleLearningrateSchedulerSelector:cosine_annealing:eta_min': 1e-08}
{'CreateImageDataLoader:batch_size': 91,
 'ImageAugmentation:augment': True,
 'ImageAugmentation:autoaugment': False,
 'ImageAugmentation:cutout': True,
 'ImageAugmentation:cutout_holes': 3,
 'ImageAugmentation:fas

23:15:31 job (1, 0, 0) failed with exception
Traceback (most recent call last):
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/hpbandster/core/worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker_no_timelimit.py", line 72, in compute
    result, randomstate = self.optimize_pipeline(config, budget, config_id, random.getstate())
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker_no_timelimit.py", line 125, in optimize_pipeline
    raise e
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker_no_timelimit.py", line 119, in optimize_pipeline
    config_id=config_id, working_directory=self.working_directory), random.getstate()
  File "/home/l/liny/an

{'CreateImageDataLoader:batch_size': 43, 'ImageAugmentation:augment': False, 'ImageAugmentation:cutout': True, 'SimpleLearningrateSchedulerSelector:lr_scheduler': 'cosine_annealing', 'SimpleTrainNode:batch_loss_computation_technique': 'standard', 'ImageAugmentation:cutout_holes': 1, 'ImageAugmentation:length': 15, 'SimpleLearningrateSchedulerSelector:cosine_annealing:T_max': 359, 'LossModuleSelectorIndices:loss_module': 'cross_entropy', 'NetworkSelectorDatasetInfo:network': 'convnet_customize', 'OptimizerSelector:optimizer': 'sgd', 'SimpleLearningrateSchedulerSelector:cosine_annealing:eta_min': 1e-08}
{'CreateImageDataLoader:batch_size': 43,
 'ImageAugmentation:augment': False,
 'ImageAugmentation:cutout': True,
 'ImageAugmentation:cutout_holes': 1,
 'ImageAugmentation:length': 15,
 'LossModuleSelectorIndices:loss_module': 'cross_entropy',
 'NetworkSelectorDatasetInfo:network': 'convnet_customize',
 'OptimizerSelector:optimizer': 'sgd',
 'SimpleLearningrateSchedulerSelector:cosine_anne

23:15:33 job (3, 0, 0) failed with exception
Traceback (most recent call last):
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/hpbandster/core/worker.py", line 206, in start_computation
    result = {'result': self.compute(*args, config_id=id, **kwargs),
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker_no_timelimit.py", line 72, in compute
    result, randomstate = self.optimize_pipeline(config, budget, config_id, random.getstate())
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker_no_timelimit.py", line 125, in optimize_pipeline
    raise e
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/core/worker_no_timelimit.py", line 119, in optimize_pipeline
    config_id=config_id, working_directory=self.working_directory), random.getstate()
  File "/home/l/liny/an

{'CreateImageDataLoader:batch_size': 34, 'ImageAugmentation:augment': True, 'ImageAugmentation:cutout': False, 'SimpleLearningrateSchedulerSelector:lr_scheduler': 'step', 'SimpleTrainNode:batch_loss_computation_technique': 'standard', 'ImageAugmentation:autoaugment': True, 'ImageAugmentation:fastautoaugment': False, 'SimpleLearningrateSchedulerSelector:step:gamma': 0.1726047402704233, 'SimpleLearningrateSchedulerSelector:step:step_size': 8, 'LossModuleSelectorIndices:loss_module': 'cross_entropy', 'NetworkSelectorDatasetInfo:network': 'convnet_customize', 'OptimizerSelector:optimizer': 'sgd', 'SimpleLearningrateSchedulerSelector:cosine_annealing:eta_min': 1e-08}
{'CreateImageDataLoader:batch_size': 34,
 'ImageAugmentation:augment': True,
 'ImageAugmentation:autoaugment': True,
 'ImageAugmentation:cutout': False,
 'ImageAugmentation:fastautoaugment': False,
 'LossModuleSelectorIndices:loss_module': 'cross_entropy',
 'NetworkSelectorDatasetInfo:network': 'convnet_customize',
 'OptimizerS

Traceback (most recent call last):
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/nodes/image/optimization_algorithm_no_timelimit.py", line 126, in fit
    res = self.parse_results(pipeline_config["result_logger_dir"])
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/autoPyTorch-0.0.2-py3.7.egg/autoPyTorch/pipeline/nodes/image/optimization_algorithm_no_timelimit.py", line 235, in parse_results
    incumbent_trajectory = res.get_incumbent_trajectory(bigger_is_better=False, non_decreasing_budget=False)
  File "/home/l/liny/anaconda3/envs/automl/lib/python3.7/site-packages/hpbandster/core/result.py", line 310, in get_incumbent_trajectory
    return_dict['config_ids'].append(return_dict['config_ids'][-1])
IndexError: list index out of range


RuntimeError: No models fit during training, please retry with a larger max_runtime.

***pred*** returns the predictions of the incumbent model. ***score*** can be used to evaluate the model on a test set. 

In [ ]:
# See how the random configuration performs (often it just predicts 0)
score = autonet.score(X_test=X_test, Y_test=Y_test)
pred = autonet.predict(X=X_test)

print("Model prediction:", pred[0:10])
print("Accuracy score", score)

Finally, you can also get the incumbent model as PyTorch Sequential model via

In [ ]:
pytorch_model = autonet.get_pytorch_model()
print(pytorch_model)